In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import urllib.request as requests

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = requests.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-12-17 16:19:36,031: INFO: common: Created dirctory at: artifacts]
[2025-12-17 16:19:36,035: INFO: common: Created dirctory at: atrifacts/data_ingestion]


[2025-12-17 16:19:43,916: INFO: 733588554: atrifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CF94:71BDD:11922B:20F27C:69428AC1
Accept-Ranges: bytes
Date: Wed, 17 Dec 2025 10:49:37 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830066-SIN
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1765968577.174656,VS0,VE711
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 3542932e39a892f87abe1fd05d1b54568e25d0ea
Expires: Wed, 17 Dec 2025 10:54:37 GMT
Source-Age: 0

]
